In [1]:
import glob, os, json
import solver
import pickle

import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.preprocessing import scale
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

import time
import math

/Users/adam/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# Paths
tspLibPath = "../data/tsplib/tsp/"
generatedPath = "../data/generated2/"

In [3]:
def loadTSPInstances(path, extension):
    instances = []
    for file in glob.glob(path + "*." + extension):
        try:
            tsp = solver.loadTSPLib(file)
            name = os.path.basename(file)
            if not tsp:
                print("Invalid file at " + name)
                continue

            tsp.setName(name)
            instances.append(tsp)
        except:
            traceback.print_exc()
    
    return instances

def loadGeneratedInstances(path):
    instances = []
    for file in glob.glob(path + "*.pytsp"):
        try:
            tspFile = open(file, "rb")
            tsp = pickle.load(tspFile)
            name = os.path.basename(file)
            
            if not tsp:
                print("Invalid file at " + name)
                continue

            tsp.setName(name)
            instances.append(tsp)
            tspFile.close()
        except:
            traceback.print_exc()
            
    return instances

In [4]:
# Load all problem instances
# tspLibInstances = loadTSPInstances(tspLibPath, "tsp")
tspLibInstances = []
generatedInstances = loadGeneratedInstances(generatedPath)

In [5]:
# adjacency = np.zeros(shape=costs.shape, dtype=np.bool)

f = open("test.adj", "w")

for row, rowArray in enumerate(costs):
    line = str(row + 1)
    for col, colValue in enumerate(rowArray):
        if colValue > 0:
            line += " " + str(col + 1)
            
    f.write(line + "\n")
        
f.close()

NameError: name 'costs' is not defined

In [6]:
savedFiles = []

def weightedEdges(instance, path):
    fullPath = path + instance.getName() + ".wel"
    f = open(fullPath, "w")
    
    savedFiles.append(fullPath)
    
    costs = instance.costs

    for row, rowArray in enumerate(costs):
        for col, colValue in enumerate(rowArray):
            if colValue > 0:
                line = str(row + 1)
                line += " " + str(col + 1)
                line += " " + str(colValue)

                f.write(line + "\n")

    f.close()

In [7]:
combinedInstances = tspLibInstances + generatedInstances
usedInstances = []

for instance in combinedInstances:
    if instance.isAsymmetric():
        continue
        
    usedInstances.append(instance)
        
    weightedEdges(instance, "../data/edges/")

In [8]:
from subprocess import call

for i in range(len(savedFiles)):
    inputFile = savedFiles[i]
    instance = usedInstances[i]
    
    result = call(["deepwalk", "--input", inputFile, "--format", "weighted_edgelist", "--output", "../data/deepwalk/" + instance.getName() + ".deep"])
    
    if result != 0:
        print("Result %d for instance %s" % (result, instance.getName()))

In [9]:
savedFiles[0]

'../data/edges/pr439.tsp.wel'

In [9]:
len(savedFiles)

498

In [10]:
len(combinedInstances)

498